# IGDA日本 SIG-for NextGeneration

```
正世話人：長久勝
副世話人：尾形美幸
　　　　　後藤誠
　　　　　湊和久
```

## ミッション

ゲーム開発者が、<font color="Red">次世代を担う若者・子供たち</font>に対して、ゲームタイトルの提供以外にできることを考え、実践する。プロのゲーム開発者との交流、ゲーム業界を支える人材の育成、ゲーム開発を通じたモノづくりや協働の体験提供などを通じ、ゲーム開発者の知見を、<font color="Red">未来の社会に還元</font>する。

## スカラーシップ

![スカラーシップ](https://i2.wp.com/www.igda.jp/wp-content/uploads/2017/09/20170830-DSC_0323.jpg)

CESAなどの支援を受け、CEDECおよびTGSでスカラーシップを提供。合わせてスタジオツアー（株式会社Aiming、株式会社サイバード、ジープラ株式会社、あまた株式会社、株式会社ディー・エヌ・エー）も実施。

## ワークショップ

![ワークショップ](https://i1.wp.com/www.igda.jp/wp-content/uploads/2017/12/20171103-DSC_0040.jpg)

中山隼雄科学技術文化財団などの支援を受け、日本全国に向けた取り組みを実施。今年度は、東京都文京区、山口県周南市、神奈川県逗子市（2/11予定）の3ヶ所。

## まだ見ぬあなたへ

![まだ見ぬあなたへ](https://auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr187/auc0303/users/5/0/2/6/rumors_r-img313x600-1433922746s8qdny14106.jpg)


# 都市雇用圏を用いたデジタルワークショップ体験機会格差の分析

中心都市とその都市に通勤する者が一定割合いる周辺地域を合わせたものを「都市雇用圏」と呼ぶ。この都市圏は、雇用のみならず、経済的な結び付き、通学など含めた生活圏を実態として良く表していると考えられる。

都市雇用圏は市区町村の集合として定義されており、圏内の市区町村でデジタルワークショップを体験する機会が提供されていれば、開催地の市区町村のみならず、圏内全域に対して機会提供されていると考えることができる。

そこで、子供向けプログラミング教室の全国的なリストをいくつか用いて、都市雇用圏ベースでの体験機会空白地特定を試みる。

## 都市雇用圏および市区町村のリストを取得



大都市雇用圏2010年基準

郊外市町村

In [ ]:
!wget http://www.csis.u-tokyo.ac.jp/UEA/MEA2010.csv

中心市町村

In [ ]:
!wget http://www.csis.u-tokyo.ac.jp/UEA/MEA2010C.csv

小都市雇用圏2010年基準

郊外市町村

In [ ]:
!wget http://www.csis.u-tokyo.ac.jp/UEA/MCEA2010.csv

中心市町村

In [ ]:
!wget http://www.csis.u-tokyo.ac.jp/UEA/MCEA2010C.csv

都市圏人口（大都市雇用圏）

In [ ]:
!wget http://www.csis.u-tokyo.ac.jp/UEA/MEA2010P.csv

都市圏人口（小都市雇用圏）

In [ ]:
!wget http://www.csis.u-tokyo.ac.jp/UEA/MCEA2010P.csv

大都市雇用圏統計データ

In [ ]:
!wget http://www.csis.u-tokyo.ac.jp/UEA/MEASTAT2010.csv

市区町村コード

In [ ]:
!wget http://www.soumu.go.jp/main_content/000442937.xls

【総計】平成29年住民基本台帳年齢階級別人口(市区町村別)

In [ ]:
!wget http://www.soumu.go.jp/main_content/000494086.xls

## データ分析準備

pandas他の必要なモジュールをインストールする。

In [ ]:
!pip install pandas xlrd jaconv beautifulsoup4 lxml chardet

市区町村リストを読み込む。

In [ ]:
import pandas as pd
import jaconv
import math

file = pd.ExcelFile("000442937.xls")
df = file.parse(file.sheet_names[0])

df = df.rename(columns={
    "都道府県名\n（漢字）": "都道府県名",
    "市区町村名\n（漢字）": "市区町村名",
    "市区町村名\n（カナ）": "カナ"
})
df = df.drop(columns=["都道府県名\n（カナ）"])
df = df.loc[:,["団体コード","市区町村名","カナ","都道府県名"]]
df = df[df["市区町村名"].isnull() == False].reset_index(drop=True)
df["カナ"] = df["カナ"].map(jaconv.h2z)
df["団体コード"] = df["団体コード"].map(lambda x: math.floor(x / 10))

municipality_list = df

print(municipality_list)

年齢階級別人口を読み込み、市区町村リストに情報を追記する。

In [ ]:
file = pd.ExcelFile("000494086.xls")
df = file.parse(file.sheet_names[0], skiprows=[0,2,3,4])
df = df.drop_duplicates(subset=["団体コード"])
df = df.rename(columns={
    "5～9": "five",
    "10～14": "ten"
})
df["団体コード"] = df["団体コード"].map(lambda x: math.floor(x / 10))

municipality_list["小中学生"] = 0
for row in df.itertuples():
    municipality_list.loc[
        municipality_list["団体コード"] == row.団体コード, "小中学生"] = row.five + row.ten

print(municipality_list)

大都市雇用圏の中心都市リストを読み込み、市区町村リストに情報を追記する。

In [ ]:
# csvの読み取り
df= pd.read_csv("MEA2010C.csv", encoding="cp932")

municipality_list["MEA"] = 0
for row in df.itertuples():
    municipality_list.loc[municipality_list["団体コード"] == row.center, "MEA"] = row.MEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.center])

小都市雇用圏の中心都市リストを読み込み、市区町村リストに情報を追記する。

In [ ]:
# csvの読み取り
df= pd.read_csv("MCEA2010C.csv", encoding="cp932")

for row in df.itertuples():
    municipality_list.loc[municipality_list["団体コード"] == row.center, "MEA"] = row.UEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.center])

大都市雇用圏の郊外都市リストを読み込み、市区町村リストに情報を追記する。

In [ ]:
# csvの読み取り
df= pd.read_csv("MEA2010.csv", encoding="cp932")

for row in df.itertuples():
    municipality_list.loc[municipality_list["団体コード"] == row.suburb, "MEA"] = row.MEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.suburb])
    municipality_list.loc[municipality_list["団体コード"] == row.suburb2, "MEA"] = row.MEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.suburb2])
    municipality_list.loc[municipality_list["団体コード"] == row.suburb3, "MEA"] = row.MEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.suburb3])
    municipality_list.loc[municipality_list["団体コード"] == row.suburb4, "MEA"] = row.MEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.suburb4])

小都市雇用圏の郊外都市リストを読み込み、市区町村リストに情報を追記する。

In [ ]:
# csvの読み取り
df= pd.read_csv("MCEA2010.csv", encoding="cp932")

for row in df.itertuples():
    municipality_list.loc[municipality_list["団体コード"] == row.suburb, "MEA"] = row.MEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.suburb])
    municipality_list.loc[municipality_list["団体コード"] == row.suburb2, "MEA"] = row.MEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.suburb2])
    municipality_list.loc[municipality_list["団体コード"] == row.suburb3, "MEA"] = row.MEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.suburb3])
    municipality_list.loc[municipality_list["団体コード"] == row.suburb4, "MEA"] = row.MEA
    print(municipality_list.loc[municipality_list["団体コード"] == row.suburb4])

都市雇用圏コードが実数になってしまうので整数型に変換する。（原因は不明なので気が向いたら調査）

In [ ]:
import numpy as np
municipality_list["MEA"] = municipality_list["MEA"].astype(np.int64)

print(municipality_list)

都市雇用圏に入らない独立した市区町村も存在する。

In [ ]:
print(municipality_list[municipality_list["MEA"] == 0])

## データ分析

市区町村リストに市区町村リストに教室の有無情報を入れるカラムを追加する。都道府県リストも作っておく。

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

municipality_list["school"] = 0

pref_list = municipality_list["都道府県名"].drop_duplicates().reset_index(drop=True)

def check_school(link) :
    print(link["pref"] + " : ", end="")
    if "pref_not_check" not in link :
        link["pref_not_check"] = False
    count = 0
    resp = requests.get(link["href"])
    resp.encoding = resp.apparent_encoding
    body_lines = resp.text.splitlines()
    for text in body_lines :
        target_municipality_list = municipality_list[
            (municipality_list["school"] == 0)
            & (municipality_list["都道府県名"] == link["pref"])
        ]
        for row in target_municipality_list.itertuples():
            if text.find("一覧") == -1 \
                and text.find(row.市区町村名) != -1 \
                and (link["pref_not_check"] or text.find(link["pref"]) != -1) :
                municipality_list.loc[municipality_list["団体コード"] == row.団体コード,
                                      "school"] = 1
                count = count + 1
                print(row.市区町村名 + ", ", end="")
    print("")
    return count

[子供向けロボット・プログラミング教室ガイド](http://xn--9ckk2d5c4051a8fm.xyz/sitemap.html#jyuusyo)から教室の情報を抽出し、市区町村リストに教室の有無情報を追記する。

In [ ]:
def check_school_guide(link) :
    if link["pref"].find("沖縄") != -1 :
        link["pref"] = "沖縄県"
    soup = BeautifulSoup(requests.get(link["href"]).content, "lxml")
    count = 0
    for end_link in soup.find_all("a") :
        if "href" in end_link.attrs.keys() \
            and end_link["href"].find(".html") != -1 \
            and end_link["href"].find("プログラミングスクール") == -1 \
            and end_link["href"].find("index") == -1 :
                if end_link["href"] == "sitemap.html" :
                    return count
                target_municipality_list = municipality_list[
                    (municipality_list["school"] == 0)
                    & (municipality_list["都道府県名"] == link["pref"])
                ]
                f = True
                for row in target_municipality_list.itertuples():
                    if end_link.get_text() == row.市区町村名 :
                        municipality_list.loc[municipality_list["団体コード"] == row.団体コード,
                                              "school"] = 1
                        count = count + 1
                        print(link["pref"] + row.市区町村名 + ", ")
                        f = False
                        break
                if f :
                    count = count + check_school(
                        {
                            "href" : "http://xn--9ckk2d5c4051a8fm.xyz/" + end_link["href"],
                            "pref" : link["pref"]
                        }
                    )
    return count

base_url = "http://xn--9ckk2d5c4051a8fm.xyz/sitemap.html"
soup = BeautifulSoup(requests.get(base_url).content, "lxml")
l = [{"href" : "http://xn--9ckk2d5c4051a8fm.xyz/" + link.find("a")["href"],
      "pref" : link.find("a").get_text()} for link in soup.find_all("p") \
         if "style" in link.attrs.keys() and link["style"].find("text-align : left;") != -1]

print("add count : " + str(sum(list(map(check_school_guide, l)))))

補完（一部の記載内容が形式的によろしくないので）

In [ ]:
municipality_list.loc[municipality_list["カナ"] == "シオガマシ", "school"] = 1
municipality_list.loc[municipality_list["カナ"] == "ミキチョウ", "school"] = 1

print(municipality_list[(municipality_list["カナ"] == "シオガマシ")
                        | (municipality_list["カナ"] == "ミキチョウ")])

[こどものミライ](http://kodomomirai.com/schools/)から教室の情報を抽出し、市区町村リストに教室の有無情報を追記する。

In [ ]:
def check_school(link) :
    print(link["pref"] + " : ", end="")
    if "pref_not_check" not in link :
        link["pref_not_check"] = False
    count = 0
    resp = requests.get(link["href"])
    resp.encoding = resp.apparent_encoding
    body_lines = resp.text.splitlines()
    for text in body_lines :
        target_municipality_list = municipality_list[
            (municipality_list["school"] == 0)
            & (municipality_list["都道府県名"] == link["pref"])
        ]
        for row in target_municipality_list.itertuples():
            if text.find(row.市区町村名) != -1 \
                and (link["pref_not_check"] or text.find(link["pref"]) != -1) :
                municipality_list.loc[municipality_list["団体コード"] == row.団体コード,
                                      "school"] = 1
                count = count + 1
                print(row.市区町村名 + ", ", end="")
    print("")
    return count

base_url = "http://kodomomirai.com/schools/"
soup = BeautifulSoup(requests.get(base_url).content, "lxml")
l = [{"href" : "http://kodomomirai.com" + link["href"],
      "pref" : re.sub("\([0-9]+\)", "", link.get_text()),
     "pref_not_check" : True}
     for link in soup.find_all("a") if link["href"].find("prefecture") != -1]

print("add count : " + str(sum(list(map(check_school, l)))))

[子供向けプログラミング教室のリスト](https://list.open-code.club/%E5%AD%90%E4%BE%9B%E5%90%91%E3%81%91%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E6%95%99%E5%AE%A4%E3%81%AE%E3%83%AA%E3%82%B9%E3%83%88/)から教室の情報を抽出し、市区町村リストに教室の有無情報を追記する。

In [ ]:
l = [{"href" : "https://list.open-code.club/子供向けプログラミング教室のリスト/" + pref + "/",
      "pref" : pref} for pref in pref_list]

print("add count : " + str(sum(list(map(check_school, l)))))

[ヒューマンアカデミーロボット教室のリスト](http://kids.athuman.com/robo/CI/Store/Synopsis2/)から教室の情報を抽出し、市区町村リストに教室の有無情報を追記する。

In [ ]:
l = [{"href" :
      "http://kids.athuman.com/robo/CI/Store/Synopsis/Search/" + \
      "?Search[Page][Offset]=1&Search[Page][DispNum]=1000&Store[store_text2]=" + pref,
      "pref" : pref, "pref_not_check" : True} for pref in pref_list]

print("add count : " + str(sum(list(map(check_school, l)))))

[アーテック エジソンアカデミーロボットプログラミング教室のリスト](http://edisonacademy.artec-kk.co.jp/robot/search.php)から教室の情報を抽出し、市区町村リストに教室の有無情報を追記する。

In [ ]:
l = [{"href" : "http://edisonacademy.artec-kk.co.jp/robot/search_city.php?pref_id=" + \
      "{0:02d}".format(i + 1),
      "pref" : pref_list[i], "pref_not_check" : True} for i in range(47)]

print("add count : " + str(sum(list(map(check_school, l)))))

独自に用意した Hour of Code 2017 日本国内会場のリスト（主催が外国人コミュニティではないと考えられるもののみ抽出）から開催地の情報を抽出し、市区町村リストに教室の有無情報を追記する。

### HoCは定常的な体験機会を作らないので、外すことにした
### csvの読み取り
df= pd.read_csv("hoc2017jp.csv", encoding="cp932")

for row in df.itertuples():
    t = municipality_list[municipality_list["都道府県名"] == row.都道府県]
    c = t[t["市区町村名"] == row.市区町村].reset_index()
    municipality_list.loc[municipality_list["団体コード"] == c.団体コード[0], "school"] = 1

print(municipality_list[municipality_list["school"] == 1])

独自に用意した Coder Dojo 日本国内会場のリストから開催地の情報を抽出し、市区町村リストに教室の有無情報を追記する。

In [ ]:
# csvの読み取り
df= pd.read_csv("cd2018.csv", encoding="cp932")

for row in df.itertuples():
    t = municipality_list[municipality_list["都道府県名"] == row.都道府県]
    c = t[t["市区町村名"] == row.市区町村].reset_index()
    municipality_list.loc[municipality_list["団体コード"] == c.団体コード[0], "school"] = 1

独自に用意した定期開催される教室やイベントの情報を、市区町村リストの教室の有無情報に追記する。

In [ ]:
# csvの読み取り
df= pd.read_csv("add_schools.csv", encoding="cp932")

for row in df.itertuples():
    t = municipality_list[municipality_list["都道府県名"] == row.都道府県]
    c = t[t["市区町村名"] == row.市区町村].reset_index()
    municipality_list.loc[municipality_list["団体コード"] == c.団体コード[0], "school"] = 1

同じ都市雇用圏に属する市区町村に体験機会がある場合、その市区町村にも体験機会があると見なし、市区町村リストに教室の有無情報を追記する。

In [ ]:
t = municipality_list[municipality_list["MEA"] != 0]
for mea in t[t["school"] == 1]["MEA"] :
    municipality_list.loc[municipality_list["MEA"] == mea, "school"] = 1

## 体験機会のない中心都市の分析

体験機会のない中心都市を抽出し、圏内の小中学生総数を求める。

In [ ]:
target_municipality_list = municipality_list[(municipality_list["school"] == 0)
                                             & (municipality_list["MEA"] == \
                                                municipality_list["団体コード"])]

target_municipality_list["圏内小中学生"] = 0
target_municipality_list["圏内自治体数"] = 0

for target in target_municipality_list.itertuples() :
    family = municipality_list[municipality_list["MEA"] == target.MEA]
    target_municipality_list.loc[target_municipality_list["団体コード"] == target.団体コード,
                                "圏内小中学生"] = sum(family["小中学生"])
    target_municipality_list.loc[target_municipality_list["団体コード"] == target.団体コード,
                                "圏内自治体数"] = len(family.index)

print(target_municipality_list.sort_values(by="圏内小中学生", ascending=False))

## 地図作成

都市雇用圏による修正を加えて、市区町村毎に体験機会の有無データを生成できたので、地図に示す。体験機会のない市区町村は赤塗で表現される。

北海道。

In [ ]:
url = "http://map.olp.yahooapis.jp/OpenLocalPlatform/V1/static?" + \
    "&width=2000&height=2000&lat=43.5&lon=142.2&z=10&" + \
    "appid=dj00aiZpPXRVRTJKRFRGb3dLeSZzPWNvbnN1bWVyc2VjcmV0Jng9MWU-&mode=blankmap&style="
for row in municipality_list[municipality_list["school"] == 0].itertuples() :
    if math.floor(row.団体コード / 1000) == 1 :
        url = url + "bm.p." + '{0:05d}'.format(row.団体コード) + ":FF0000|"

print(url)

東北。

In [ ]:
url = "http://map.olp.yahooapis.jp/OpenLocalPlatform/V1/static?" + \
    "&width=2000&height=2000&lat=39.5&lon=140.2&z=10&" + \
    "appid=dj00aiZpPXRVRTJKRFRGb3dLeSZzPWNvbnN1bWVyc2VjcmV0Jng9MWU-&mode=blankmap&style="
for row in municipality_list[municipality_list["school"] == 0].itertuples() :
    if math.floor(row.団体コード / 1000) >= 2 and math.floor(row.団体コード / 1000) <= 16 :
        url = url + "bm.p." + '{0:05d}'.format(row.団体コード) + ":FF0000|"

print(url)

関東甲信越、北陸、中部。

In [ ]:
url = "http://map.olp.yahooapis.jp/OpenLocalPlatform/V1/static?" + \
    "&width=2000&height=2000&lat=35.5&lon=138.2&z=10&" + \
    "appid=dj00aiZpPXRVRTJKRFRGb3dLeSZzPWNvbnN1bWVyc2VjcmV0Jng9MWU-&mode=blankmap&style="
for row in municipality_list[municipality_list["school"] == 0].itertuples() :
    if math.floor(row.団体コード / 1000) >= 7 and math.floor(row.団体コード / 1000) <= 30 :
        url = url + "bm.p." + '{0:05d}'.format(row.団体コード) + ":FF0000|"

print(url)

近畿、中国、四国。

In [ ]:
url = "http://map.olp.yahooapis.jp/OpenLocalPlatform/V1/static?" + \
    "&width=2000&height=2000&lat=33.5&lon=132.9&z=10&" + \
    "appid=dj00aiZpPXRVRTJKRFRGb3dLeSZzPWNvbnN1bWVyc2VjcmV0Jng9MWU-&mode=blankmap&style="
for row in municipality_list[municipality_list["school"] == 0].itertuples() :
    if math.floor(row.団体コード / 1000) >= 18 and math.floor(row.団体コード / 1000) <= 46 :
        url = url + "bm.p." + '{0:05d}'.format(row.団体コード) + ":FF0000|"

print(url)

九州。

In [ ]:
url = "http://map.olp.yahooapis.jp/OpenLocalPlatform/V1/static?" + \
    "&width=2000&height=2000&lat=32.0&lon=130.0&z=10&" + \
    "appid=dj00aiZpPXRVRTJKRFRGb3dLeSZzPWNvbnN1bWVyc2VjcmV0Jng9MWU-&mode=blankmap&style="
for row in municipality_list[municipality_list["school"] == 0].itertuples() :
    if math.floor(row.団体コード / 1000) >= 34 and math.floor(row.団体コード / 1000) <= 46 :
        url = url + "bm.p." + '{0:05d}'.format(row.団体コード) + ":FF0000|"

print(url)

沖縄。

In [ ]:
url = "http://map.olp.yahooapis.jp/OpenLocalPlatform/V1/static?" + \
    "&width=2000&height=2000&lat=27.0&lon=126.0&z=9&" + \
    "appid=dj00aiZpPXRVRTJKRFRGb3dLeSZzPWNvbnN1bWVyc2VjcmV0Jng9MWU-&mode=blankmap&style="
for row in municipality_list[municipality_list["school"] == 0].itertuples() :
    if math.floor(row.団体コード / 1000) >= 45 :
        url = url + "bm.p." + '{0:05d}'.format(row.団体コード) + ":FF0000|"

print(url)

## 分析結果

体験機会のない市区町村。

In [ ]:
print(municipality_list[municipality_list["school"] == 0])

千葉県と大分県と岡山県。

In [ ]:
print(municipality_list[(municipality_list["school"] == 0)
                        & (municipality_list["都道府県名"] == "千葉県")])
print(municipality_list[(municipality_list["school"] == 0)
                        & (municipality_list["都道府県名"] == "大分県")])
print(municipality_list[(municipality_list["school"] == 0)
                        & (municipality_list["都道府県名"] == "岡山県")])

体験のない雇用圏。

In [ ]:
print(target_municipality_list.sort_values(by="圏内小中学生", ascending=False))

地図。

![北海道](hokkaido.png)
![東北](tohoku.png)
![関東](kanto.png)
![関西](kansai.png)
![九州](kyushu.png)
![沖縄](okinawa.png)

## さいごに

教室リストの網羅精度をより高めれば、空白地帯を把握することが可能になると思われるが、現在の結果を見るところ、少し精度が低い。

この結果を手掛かりに、地方での体験機会創出に繋げたい。

# まだ見ぬあなたへ

![まだ見ぬあなたへ](https://auctions.c.yimg.jp/images.auctions.yahoo.co.jp/image/dr187/auc0303/users/5/0/2/6/rumors_r-img313x600-1433922746s8qdny14106.jpg)